<a href="https://colab.research.google.com/github/bimsarapathiraja/datastorm/blob/master/day3/lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive') #mounting gdrive   

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive

/content/gdrive/My Drive


In [0]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler,Normalizer #Normalizer scaler was the best option

In [0]:
data = pd.read_csv('data-storm-10/credit_card_default_train.csv') #loading the training dataset

In [0]:
X = data.drop('NEXT_MONTH_DEFAULT', 1)
y = data['NEXT_MONTH_DEFAULT']

X = X.drop('Client_ID', 1)

xdata = pd.read_csv('data-storm-10/credit_card_default_test.csv')
test = xdata.drop('Client_ID', 1)
test_id = xdata['Client_ID']

In [0]:
data_comp = X.append(test) #combining training and test
data_comp.shape

(30000, 23)

In [0]:
data_comp['Balance_Limit_V1'] = data_comp['Balance_Limit_V1'].map({'100K': 100000, '200K': 200000, '300K': 300000, '400K': 400000, ' 500K': 500000, '1M': 1000000, '1.5M': 1500000, '2.5M': 2500000})
#mapping values

In [0]:
data_comp[['Balance_Limit_V1', 'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV',
       'PAY_DEC', 'DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP', 'DUE_AMT_OCT',
       'DUE_AMT_NOV', 'DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG',
       'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']] = preprocessing.scale(data_comp[['Balance_Limit_V1', 'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT', 'PAY_NOV',
       'PAY_DEC', 'DUE_AMT_JULY', 'DUE_AMT_AUG', 'DUE_AMT_SEP', 'DUE_AMT_OCT',
       'DUE_AMT_NOV', 'DUE_AMT_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG',
       'PAID_AMT_SEP', 'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC']]) #scaling

In [0]:
data_comp = pd.get_dummies(data_comp,columns=['Gender', 'EDUCATION_STATUS', 'MARITAL_STATUS',
       'AGE']) #one hot encoding

In [0]:
def find_correlation(data, threshold=0.85): #removing correlated rows > 0.85
    corr_mat = data.corr()
    corr_mat.loc[:, :] = np.tril(corr_mat, k=-1)
    already_in = set()
    result = []
    for col in corr_mat:
        perfect_corr = corr_mat[col][abs(corr_mat[col])> threshold].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)
    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

In [0]:
columns_to_drop = find_correlation(data_comp , 0.85)
print(columns_to_drop)
data_comp = data_comp.drop(columns=columns_to_drop) # columns are dropped

['DUE_AMT_SEP', 'DUE_AMT_OCT', 'DUE_AMT_JULY', 'DUE_AMT_NOV', 'Gender_F', 'MARITAL_STATUS_Other']


In [0]:
X = data_comp.iloc[:len(X)]
test = data_comp.iloc[len(X):] 

In [0]:
X.shape

(24000, 24)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2) #train test dataset splitting

In [0]:
from sklearn.linear_model import LogisticRegression #trying different models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
#from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

classifiers=[['Logistic Regression :',LogisticRegression(max_iter = 1500)],
       ['Decision Tree Classification :',DecisionTreeClassifier()],
       ['Gradient Boosting Classification :', GradientBoostingClassifier()],
       ['Ada Boosting Classification :',AdaBoostClassifier()],
       ['RandomForest Classification :',AdaBoostClassifier()],
       ['Extra Tree Classification :', ExtraTreesClassifier()],
       ['K-Neighbors Classification :',KNeighborsClassifier()],
       ['Support Vector Classification :',SVC()],
       ['Gaussian Naive Bayes :',GaussianNB()]]
cla_pred=[]
for name,model in classifiers:
    model=model
    model.fit(X_train,y_train)
    predictions = model.predict(X_test)
    cla_pred.append(accuracy_score(y_test,predictions))
    print(name,accuracy_score(y_test,predictions))

Logistic Regression : 0.8160416666666667
Decision Tree Classification : 0.7197916666666667
Gradient Boosting Classification : 0.8252083333333333
Ada Boosting Classification : 0.8254166666666667
RandomForest Classification : 0.8254166666666667
Extra Tree Classification : 0.8027083333333334
K-Neighbors Classification : 0.7925
Support Vector Classification : 0.8220833333333334
Gaussian Naive Bayes : 0.6560416666666666


In [0]:
#LGBMClassifier

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def lgbm_evaluate(**params):
#     warnings.simplefilter('ignore')
    
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth'])
        
    clf = LGBMClassifier(**params, n_estimators=20000, nthread=-1)

    test_pred_proba = np.zeros((X_train.shape[0], 2))
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X_train, y_train)):
        X_train_bo, X_valid = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_train_bo, y_valid = y_train.iloc[train_idx], y_train.iloc[valid_idx]
        
        model = LGBMClassifier(**params, n_estimators = 10000, n_jobs = -1)
        model.fit(X_train_bo, y_train_bo, 
                eval_set=[(X_train_bo, y_train_bo), (X_valid, y_valid)], eval_metric='binary_logloss',
                verbose=False, early_stopping_rounds=200)

        y_pred_valid = model.predict_proba(X_valid)

        test_pred_proba[valid_idx] = y_pred_valid

    return accuracy_score(y_valid, y_pred_valid.argmax(1))

In [0]:
#hyper parameter tuning
params = {'colsample_bytree': (0.6, 1),
     'learning_rate': (.001, .08), 
      'num_leaves': (8, 124), 
      'subsample': (0.6, 1), 
      'max_depth': (3, 25), 
      'reg_alpha': (.05, 15.0), 
      'reg_lambda': (.05, 15.0), 
      'min_split_gain': (.001, .03),
      'min_child_weight': (12, 80)}

from sklearn.model_selection import StratifiedKFold
n_fold = 20
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=11)

from bayes_opt import BayesianOptimization
bo = BayesianOptimization(lgbm_evaluate, params)
bo.maximize(init_points=5, n_iter=5)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.8292   |  0.9571   |  0.01499  |  9.261    |  76.82    |  0.02275  |  32.33    |  0.7073   |  4.773    |  0.8926   |
|  2        |  0.8344   |  0.826    |  0.03198  |  20.51    |  76.84    |  0.02918  |  69.39    |  14.94    |  10.9     |  0.7892   |
|  3        |  0.8344   |  0.8477   |  0.01537  |  14.3     |  59.86    |  0.001263 |  86.84    |  11.31    |  11.67    |  0.76     |
|  4        |  0.8292   |  0.721    |  0.03523  |  12.55    |  50.48    |  0.009645 |  21.43    |  3.242    |  8.533    |  0.7408   |
|  5        |  0.8302   |  0.7722   |  0.06408  |  14.42    |  53.68    |  0.01189  |  77.8     |  12.93    |  14.23    |  0.9604   |
|  6        |  0.8313   |  0.837    |  0.0757   |  21.54    | 

In [0]:
params = {'num_leaves': int(bo.max['params']['num_leaves']), #parameter finding
          'min_data_in_leaf': int(bo.max['params']['min_child_weight']),
          'min_split_gain': bo.max['params']['min_split_gain'],
          'objective': 'binary',
          'max_depth': int(bo.max['params']['max_depth']),
          'learning_rate': bo.max['params']['learning_rate'],
          "boosting": "gbdt",
          "bagging_freq": 5,
          "bagging_fraction": bo.max['params']['subsample'],
          "bagging_seed": 11,
          "verbosity": -1,
          'reg_alpha': bo.max['params']['reg_alpha'],
          'reg_lambda': bo.max['params']['reg_lambda'],
          "num_class": 1,
          'nthread': -1
         }

In [0]:
params

# Calculated parameters ------------------------------------------------------------------------------

# {'bagging_fraction': 0.7891682295772582,
#  'bagging_freq': 5,
#  'bagging_seed': 11,
#  'boosting': 'gbdt',
#  'learning_rate': 0.031982900805891605,
#  'max_depth': 20,
#  'min_data_in_leaf': 76,
#  'min_split_gain': 0.02917582230097086,
#  'nthread': -1,
#  'num_class': 1,
#  'num_leaves': 69,
#  'objective': 'binary',
#  'reg_alpha': 14.937925929518842,
#  'reg_lambda': 10.899395919007185,
#  'verbosity': -1}

#--------------------------------------------------------------------------------------------------------

{'bagging_fraction': 0.7891682295772582,
 'bagging_freq': 5,
 'bagging_seed': 11,
 'boosting': 'gbdt',
 'learning_rate': 0.031982900805891605,
 'max_depth': 20,
 'min_data_in_leaf': 76,
 'min_split_gain': 0.02917582230097086,
 'nthread': -1,
 'num_class': 1,
 'num_leaves': 69,
 'objective': 'binary',
 'reg_alpha': 14.937925929518842,
 'reg_lambda': 10.899395919007185,
 'verbosity': -1}

In [0]:
from lightgbm import LGBMClassifier #model fitting
from sklearn.metrics import accuracy_score

model = LGBMClassifier(**params, n_estimators = 10000, n_jobs = -1)
model.fit(X_train, y_train, 
        eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric='binary_logloss',
        verbose=5000, early_stopping_rounds=4000)

predictions = model.predict(X_test)
predictions_ = model.predict(X_train)

print(accuracy_score(y_test, predictions), accuracy_score(y_train, predictions_))

Training until validation scores don't improve for 4000 rounds.
Early stopping, best iteration is:
[188]	training's binary_logloss: 0.412985	training's binary_logloss: 0.412985	valid_1's binary_logloss: 0.437175	valid_1's binary_logloss: 0.437175
0.8127083333333334 0.8246875


In [0]:
from sklearn.metrics import classification_report,confusion_matrix #evaluating the model
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test,predictions))

[[3508  197]
 [ 702  393]]
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      3705
           1       0.67      0.36      0.47      1095

    accuracy                           0.81      4800
   macro avg       0.75      0.65      0.68      4800
weighted avg       0.80      0.81      0.79      4800

0.8127083333333334


In [0]:
pred = model.predict(test) #saving predictions
submissionlgbm = pd.DataFrame({
    'Client_ID': test_id,
    'NEXT_MONTH_DEFAULT': pred
})
print(submissionlgbm.NEXT_MONTH_DEFAULT.value_counts())
submissionlgbm.to_csv('submissionlgbm3.csv', index = False)

0    5330
1     670
Name: NEXT_MONTH_DEFAULT, dtype: int64
